## Function to calculate BMI  z-score's for children 

LMS calculation for the BMI z-score 

Link to formulas used in the function: https://cdn.who.int/media/docs/default-source/child-growth/growth-reference-5-19-years/computation.pdf?sfvrsn=c2ff6a95_4 

- All calculations are with age(months), weight(kg), height(cm)

- The WHO reference sheets will be downloaded automatically from github as needed. 

- You must have following columns named in the correct format "age", "sex", "weight", "height". 

- You must NOT have a column named "BMI" in your intial dataframe

- The function will create three new columns: BMI, reference sheet used, and z-score




OPTIONS 
- Age in months is defualt, but if you have it in years use following argument "age_in_years = True" 
- Month 61 is default from 5-19 reference to change to 0-5 use following argument "month61_from_0to5_reference = True"

Now you can load the function and go! 



## BMI z-score(WHO) function 

In [1]:
# Needed imports

import pandas as pd
import math

In [2]:
## Create WHO z-scores for children 0-19 years old
def zscore(df, age_in_years = False, month61_from_0to5_reference = False): 
    
    #ARGUMENT: age_in_years = True
    if age_in_years:
        df['age(months)'] = df['age']*12
   #age_in_years = False 
    else:
        df['age(months)'] = df['age']        
        
        
        
    # create BMI column 
    weight = df['weight']
    height = (df['height']/100)
    BMI = []
    BMI = weight/(height**2)
    df['BMI'] = round(BMI,2)

    # this is the criteria for above/below 5 years in months (60 months = 5 years )
    age_5years = 60             

    # this is a for-loop to iterate through every single line of the dataframe 
    for index, row in df.iterrows():

    # create new columns to save in 
        df.at[index, 'reference_sheet'] = ""
        df.at[index, 'zscore'] = ""
        
    # save parameters for calculations    
        reference_sheet = []
        age = row['age(months)']            
        sex = row['sex']
        BMI = row['BMI'] # this one is for the z-score calculation 
        
    # references from Github
        boy_below_5 = "https://raw.githubusercontent.com/molne1/WHO_z-score_children_0-19/main/WHO_reference_data/WHO_boy_reference_below_5_years.csv"
        girl_below_5 ="https://raw.githubusercontent.com/molne1/WHO_z-score_children_0-19/main/WHO_reference_data/WHO_girl_reference_below_5_years.csv"
        boy_above_5 = "https://raw.githubusercontent.com/molne1/WHO_z-score_children_0-19/main/WHO_reference_data/WHO_boy_reference_above_5_years.csv"
        girl_above_5 ="https://raw.githubusercontent.com/molne1/WHO_z-score_children_0-19/main/WHO_reference_data/WHO_girl_reference_above_5_years.csv"
    
    
    
    # Decide what reference material and prepare to get the values you need from it (boy/girl, above/below 5years)     
    
    # ARGUMENT: month61_from_0to5_reference = True
        
        if month61_from_0to5_reference == True:              # if condition is true 
            if row['age(months)'] == 61:                     # check if child is 61 months 
                if sex == 'boy':                              # if it is a boy get this reference
                    reference = pd.read_csv(boy_below_5)      # get from github
                    reference_sheet = ['Boy below 5_forced']  # Tell us what reference sheet I got
                else:                                          # If it is a girl - get this 
                    reference = pd.read_csv(girl_below_5)
                    reference_sheet = ['Girl below 5_forced']
                    
     # ARGUMENT(standard): month61_from_0to5_reference = False                                    
            else:
                if age > age_5years:
                    if sex == 'boy':
                        reference = pd.read_csv(boy_above_5)
                        reference_sheet = ['Boy above 5']                        
                    else: #girl
                        reference = pd.read_csv(girl_above_5)
                        reference_sheet = ['Girl above 5']


                else:
                    if sex == 'boy':
                        reference = pd.read_csv(boy_below_5)
                        reference_sheet = ["Boy below 5"]
                    else: #girl
                        reference = pd.read_csv(girl_below_5)
                        reference_sheet = ["Girl below 5"]
                
                
            
        else:
            if age > age_5years:
                if sex == 'boy':
                    reference = pd.read_csv(boy_above_5)
                    reference_sheet = ['Boy above 5']                        
                else: #girl 
                    reference = pd.read_csv(girl_above_5)
                    reference_sheet = ['Girl above 5']


            else:
                if sex == 'boy':
                    reference = pd.read_csv(boy_below_5)
                    reference_sheet = ["Boy below 5"]
                else: #girl 
                    reference = pd.read_csv(girl_below_5)
                    reference_sheet = ["Girl below 5"]
                

                
    # Extract LMS values 
        individual_reference = reference[reference["Months"] == age]
        L = float(individual_reference["L"].iloc[0])              
        M = float(individual_reference["M"].iloc[0])
        S = float(individual_reference["S"].iloc[0]) 

        SD2pos = (M*((1+L*S*2)** (1 / L)))
        SD3pos = (M*((1+L*S*3)** (1 / L)))
        SD23pos= (SD3pos) - (SD2pos)

        SD2neg = (M*((1+L*S*(-2))** (1 / L)))
        SD3neg = (M*((1+L*S*(-3))** (1 / L)))
        SD23neg= (SD2neg)- (SD3neg)


    # Calculate z-score and apply corrections 
        zscore = (((BMI/M)**L) -1)/(S*L)
        if zscore > 3:
            zscore = 3+((BMI-SD3pos)/SD23pos)
        if zscore <-3:
            zscore = -3+((BMI-SD3neg)/SD23neg)
        

            
    # save columns with reference sheet used and z-score calculated 
        df.at[index, 'reference_sheet'] = reference_sheet
        df.at[index, 'zscore'] = round(zscore,2)
        df['zscore'] = pd.to_numeric(df['zscore'])
        
    return df

### Test data - delete if you are using script 

Test data is from examples in WHO documentation for 5-19 years old with the addition of 61 months(which excists in both references) and 2 children below 5. 

- Height and weight are made up for all individuals ( for child 1-3, it was made to correspond to exampel BMI) 
- All numbers corresponing to 99 are made up 
- Child 4-6 have made up zcontrol numvers to test rounding function (rounding to 0.01 decimals ) 

In [3]:
# Load data of example individuals.
data = {'example': ['child 1', 'child 2', 'child 3', 'child 4: 61 months', 'child 5: girl below 5', "child 6: boy below 5"],
        'age(years)':[11, 16, 9, 5, 1, 1],
        'age':[132, 192, 108, 61, 12, 12],
        'sex': ['boy','boy','boy', 'girl', 'girl', 'boy'],
        'actual_BMI':[30,14,19,99,99, 99],
        'weight':[81.68, 38.12 ,51.73, 15, 8, 10],
        'height':[165, 165, 165, 60, 80, 50],
        'L':[-1.7862, -1.3529, -1.6318, 99, 99, 99],
       'M':[16.9392, 20.4951, 16.0490,99, 99, 99],
        'S':[0.11070, 0.12579, 0.10038,99, 99, 99],
        'zcontrol':[ 3.35, -3.8, 1.47,11.21, 12.36, 13.16]}

# Create DataFrame
df = pd.DataFrame(data)
df.dtypes

example        object
age(years)      int64
age             int64
sex            object
actual_BMI      int64
weight        float64
height          int64
L             float64
M             float64
S             float64
zcontrol      float64
dtype: object

In [4]:
# Test function without any conditions 
zscore(df)

#produce test results 
for index, row in df.iterrows(): 
    if math.isclose(row['zcontrol'] - row['zscore'], 0.0, abs_tol=0.01):
        df.at[index, 'test passed'] = "passed"
    else:
        df.at[index, 'test passed'] = "failed"
df

,example,age(years),age,sex,actual_BMI,weight,height,L,M,S,zcontrol,age(months),BMI,reference_sheet,zscore,test passed
0,child 1,11,132,boy,30,81.68,165,-1.7862,16.9392,0.11070,3.35,132,30.00,[Boy above 5],3.35,passed
1,child 2,16,192,boy,14,38.12,165,-1.3529,20.4951,0.12579,-3.80,192,14.00,[Boy above 5],-3.79,passed
2,child 3,9,108,boy,19,51.73,165,-1.6318,16.0490,0.10038,1.47,108,19.00,[Boy above 5],1.47,passed
3,child 4: 61 months,5,61,girl,99,15.00,60,99.0000,99.0000,99.00000,11.21,61,41.67,[Girl above 5],11.19,failed
4,child 5: girl below 5,1,12,girl,99,8.00,80,99.0000,99.0000,99.00000,12.36,12,12.50,[Girl below 5],-3.20,failed
5,child 6: boy below 5,1,12,boy,99,10.00,50,99.0000,99.0000,99.00000,13.16,12,40.00,[Boy below 5],13.15,passed


In [5]:
# Test function with change of reference for 61 months --> see change in reference sheet used
zscore(df,age_in_years = False, month61_from_0to5_reference = True )

#produce test results 
for index, row in df.iterrows(): 
    if math.isclose(row['zcontrol'] - row['zscore'], 0.0, abs_tol=0.01):
        df.at[index, 'test passed'] = "passed"
    else:
        df.at[index, 'test passed'] = "failed"
df

,example,age(years),age,sex,actual_BMI,weight,height,L,M,S,zcontrol,age(months),BMI,reference_sheet,zscore,test passed
0,child 1,11,132,boy,30,81.68,165,-1.7862,16.9392,0.11070,3.35,132,30.00,[Boy above 5],3.35,passed
1,child 2,16,192,boy,14,38.12,165,-1.3529,20.4951,0.12579,-3.80,192,14.00,[Boy above 5],-3.79,passed
2,child 3,9,108,boy,19,51.73,165,-1.6318,16.0490,0.10038,1.47,108,19.00,[Boy above 5],1.47,passed
3,child 4: 61 months,5,61,girl,99,15.00,60,99.0000,99.0000,99.00000,11.21,61,41.67,[Girl below 5_forced],12.04,failed
4,child 5: girl below 5,1,12,girl,99,8.00,80,99.0000,99.0000,99.00000,12.36,12,12.50,[Girl below 5],-3.20,failed
5,child 6: boy below 5,1,12,boy,99,10.00,50,99.0000,99.0000,99.00000,13.16,12,40.00,[Boy below 5],13.15,passed
